In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

#load data

#Lineares Model fitting
# Fit the linear model
mod = smf.ols('price ~ sqft_lot15 + C(condition)', data=house_pricing).fit()

# Print the summary
print(mod.summary())